Una red social almacena el contenido de los chats entre sus usuarios
en un RDD que tiene registros con el siguiente formato: (chat_id,
user_id, nickname, text). Queremos saber cuál es el usuario (user_id)
que mas preguntas hace en sus chats, contabilizamos una pregunta por
cada caracter “?” que aparezca en el campo text. Programar en Spark
un programa que identifique al usuario preguntón.  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqrrr2h4xh)

Vamos a usar el guion de "La isla desierta" de Roberto Arlt como unico documento para armar el RDD. En la siguiente celda proceso el guion, que se encuentra en la carpeta textos.

In [ ]:
lineas = []
personajes = set([])
with open("textos/la_isla_desierta.txt","r") as f:
    for linea in f:
        if ".-" not in linea: continue
        lineas.append(linea.split(".-"))
        personajes.add(lineas[-1][0].strip())
ids_personajes = {nombre:num for num,nombre in enumerate(personajes)}
guion = [(1,ids_personajes[x[0]],x[0],x[1].strip()) for x in lineas]

Tenemos el guion procesado, lo cual nos da registros como los del enunciado:  
```(chat_id, user_id, nickname, text)```  
Lo primero que hacemos es averiguar cuantas preguntas hay en cada linea del guion y quien la dice. Usamos [string.count](https://docs.python.org/2/library/string.html#string.count) para saber la cantidad de veces que aparece `?` en cada linea.  
Luego con reduceByKey agrupamos por nickname y sumamos la cantidad de veces que hizo preguntas.  
Con [rdd.max](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.max) obtenemos el registro con mayor cantidad de preguntas, definiendo una funcion que tome la cantidad de preguntas para ordenar.

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

rdd = sc.parallelize(guion)
rdd = rdd.map(lambda x: (x[2], x[3].count("?")))
rdd = rdd.reduceByKey(lambda x, y: x+y)
user = rdd.max(key=lambda x: x[1])[0]

In [ ]:
"El usuario que mas preguntas hizo fue {}".format(user)